In [1]:
import torch

# GPU 지원 여부 확인
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
else:
    print("GPU is not available. Using CPU.")


GPU is available: Tesla T4
Number of GPUs available: 1


In [2]:
pwd

'/home/ec2-user/SageMaker/yujin'

In [2]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix



In [3]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
# 작업 디렉터리 및 데이터 경로 설정
base_dir = "/home/ec2-user/SageMaker"
data_dir = os.path.join(base_dir, "data", "Final_Data")
ok_dir = os.path.join(data_dir, "OK")
ng_dir = os.path.join(data_dir, "NG")

# 분할된 데이터 저장 경로
split_data_dir = os.path.join(base_dir, "Data", "Binary")
train_dir = os.path.join(split_data_dir, "train")
val_dir = os.path.join(split_data_dir, "val")
test_dir = os.path.join(split_data_dir, "test")

# 분할 비율
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 데이터 가져오기
ok_images = [os.path.join(ok_dir, f) for f in os.listdir(ok_dir) if f.lower().endswith(".bmp")]
ng_images = [os.path.join(ng_dir, f) for f in os.listdir(ng_dir) if f.lower().endswith(".bmp")]

# 데이터 섞기
random.shuffle(ok_images)
random.shuffle(ng_images)

# 데이터 분할 함수
def split_data(data_list, train_ratio, val_ratio):
    train, temp = train_test_split(data_list, test_size=(1 - train_ratio))
    val, test = train_test_split(temp, test_size=(test_ratio / (test_ratio + val_ratio)))
    return train, val, test

# OK와 NG 데이터 분할
ok_train, ok_val, ok_test = split_data(ok_images, train_ratio, val_ratio)
ng_train, ng_val, ng_test = split_data(ng_images, train_ratio, val_ratio)

# 파일 복사 함수
def copy_files(file_list, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for file_path in file_list:
        shutil.copy(file_path, target_dir)

# 분할된 데이터를 저장
def save_split_data(class_name, train_files, val_files, test_files):
    copy_files(train_files, os.path.join(train_dir, class_name))
    copy_files(val_files, os.path.join(val_dir, class_name))
    copy_files(test_files, os.path.join(test_dir, class_name))

save_split_data("OK", ok_train, ok_val, ok_test)
save_split_data("NG", ng_train, ng_val, ng_test)

print(f"데이터 분할 완료! 분할된 데이터는 {split_data_dir}에 저장되었습니다.")

ValueError: With n_samples=0, test_size=0.30000000000000004 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [4]:
# 데이터 전처리 및 변환
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# DataLoader 설정
dataloaders = {
    x: DataLoader(
        datasets.ImageFolder(root=os.path.join(split_data_dir, x), transform=data_transforms[x]),
        batch_size=32, shuffle=(x == 'train'), num_workers=4
    )
    for x in ['train', 'val', 'test']
}

dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val', 'test']}
class_names = dataloaders['train'].dataset.classes

print(f"Classes: {class_names}")
print(f"Dataset sizes: {dataset_sizes}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/yujin/Raw_Data_Split/train'

In [12]:
# 모델 정의 및 학습

In [ ]:
# 모델 정의 (Pre-trained ResNet50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

# Output Layer 수정 (OK/NG 이진 분류)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()  # 이진 분류
)
model = model.to(device)

# 손실 함수와 옵티마이저
criterion = nn.BCELoss()  # Binary CrossEntropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 및 검증 함수
def train_model(model, criterion, optimizer, dataloaders, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = (outputs > 0.5).float()
                    loss = criterion(outputs.squeeze(), labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)
    return model

# 모델 학습
model = train_model(model, criterion, optimizer, dataloaders, num_epochs=20)


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 220MB/s]


Epoch 1/25
train Loss: 0.1360 Acc: 29.3082
val Loss: 0.3130 Acc: 28.2239
Epoch 2/25
train Loss: 0.0924 Acc: 29.1026
val Loss: 0.5078 Acc: 26.2697
Epoch 3/25
train Loss: 0.0723 Acc: 28.9324
val Loss: 0.0704 Acc: 30.5980
Epoch 4/25
train Loss: 0.0614 Acc: 28.9231
val Loss: 0.1183 Acc: 30.4682
Epoch 5/25
train Loss: 0.0673 Acc: 28.8762
val Loss: 0.0507 Acc: 30.7430
Epoch 6/25
train Loss: 0.0525 Acc: 28.8063
val Loss: 0.0447 Acc: 30.8244
Epoch 7/25
train Loss: 0.0520 Acc: 28.7845
val Loss: 0.0634 Acc: 30.6718
Epoch 8/25
train Loss: 0.0506 Acc: 28.8216
val Loss: 0.0457 Acc: 30.7837
Epoch 9/25
train Loss: 0.0434 Acc: 28.7927
val Loss: 0.0977 Acc: 30.1756
Epoch 10/25
train Loss: 0.0466 Acc: 28.7725
val Loss: 0.0492 Acc: 30.5980
Epoch 11/25
train Loss: 0.0355 Acc: 28.7654
val Loss: 0.0451 Acc: 30.6616
Epoch 12/25
train Loss: 0.0266 Acc: 28.7485
val Loss: 0.0866 Acc: 30.4173
Epoch 13/25
train Loss: 0.0313 Acc: 28.7289
val Loss: 0.0686 Acc: 30.6081
Epoch 14/25
train Loss: 0.0258 Acc: 28.6896
val

In [14]:
# 테스트 데이터 평가

In [16]:
import time
from sklearn.metrics import classification_report, confusion_matrix, f1_score

def evaluate_model(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []

    # 추론 시간 측정 시작
    start_time = time.time()

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs)
            preds = (outputs > 0.5).float()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # 추론 시간 측정 종료
    end_time = time.time()
    inference_time = end_time - start_time

    # Confusion Matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # F1-score 점수 출력
    f1 = f1_score(y_true, y_pred, average="weighted")
    print(f"\nWeighted F1-score: {f1:.4f}")

    # 추론 시간 출력
    print(f"\nInference Time (Total): {inference_time:.4f} seconds")
    print(f"Average Inference Time per Sample: {inference_time / len(y_true):.4f} seconds")

# 테스트 데이터 평가
evaluate_model(model, dataloaders['test'])



Confusion Matrix:
[[ 27  18]
 [  2 740]]

Classification Report:
              precision    recall  f1-score   support

          NG       0.93      0.60      0.73        45
          OK       0.98      1.00      0.99       742

    accuracy                           0.97       787
   macro avg       0.95      0.80      0.86       787
weighted avg       0.97      0.97      0.97       787


Weighted F1-score: 0.9720

Inference Time (Total): 34.5903 seconds
Average Inference Time per Sample: 0.0440 seconds
